# Explore triggered events

In [ ]:
import os
import sys

import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import plotly.io as pio
import tqdm.notebook as tqdm

from plotting.hit_plots import plot_slices, plot_3d, plotly_3d
from line_from_planes import LineFromPlanes
from local_config import get_triggered_hits, img_path


pio.renderers.default = "chromium"  # You might want to change this.

In [ ]:
cut = " nhit_slab > 10 "
min_valid_hits_in_event = 10

In [ ]:
events_all = get_triggered_hits(cut)
print("Hit vars:", ", ".join([f for f in events_all.fields if f.startswith("hit_")]))


def trigger_str(is_triggered):
    return (
        f"{sum(ak.sum(is_triggered, axis=1) > 0)} events "
        f"with {ak.sum(is_triggered)} valid channel hits "
        f"({len(is_triggered)} candidate events)."
    )


is_triggered = (events_all.hit_isHit == 1) & (events_all.hit_isMasked == 0)
print(cut + ": \n\t" + trigger_str(is_triggered))
is_triggered = is_triggered & (ak.sum(is_triggered, axis=1) >= min_valid_hits_in_event)
print(f"Additionally require >={min_valid_hits_in_event} valid channel hits: ")
print("\t" + trigger_str(is_triggered))
is_triggered = is_triggered & (events_all.hit_energy > 0)
print("Only positive energy: ")
print("\t" + trigger_str(is_triggered))

events = events_all[ak.any(is_triggered, axis=1)]
is_triggered = is_triggered[ak.any(is_triggered, axis=1)]
x = events.hit_x[is_triggered]
y = events.hit_y[is_triggered]
z = events.hit_z[is_triggered]
e = events.hit_energy[is_triggered]

x_pos = np.array(sorted(set(ak.flatten(events.hit_x))))
y_pos = np.array(sorted(set(ak.flatten(events.hit_y))))
z_pos = np.array(sorted(set(ak.flatten(events.hit_z))))
pos = {"x": x_pos, "y": y_pos, "z": z_pos}

# Explore triggered events

In [ ]:
def channel_remapping(dim):
    # There seems to be an issue in some layers.
    xy_mirrored = np.any([dim["z"] == mir for mir in [0, 2, 4, 9, 10]], axis=0)
    dim["x"] = dim["x"].copy()
    dim["y"] = dim["y"].copy()
    dim["x"][xy_mirrored] *= -1
    dim["y"][xy_mirrored] *= -1
    return dim


def do_plotting(i, unmasked=False, remapping=False, only_slices=False):
    if unmasked:
        ev = events_all[events_all.event == events[i].event][0]
        xyz_dict = dict(x=ev.hit_x, y=ev.hit_y, z=ev.hit_z, e=ev.hit_energy)
        dim = {k: var.to_numpy() for k, var in xyz_dict.items()}
        # Here, you can patch in some selection criteria.
        # dim["e"][dim["e"] < 0] = 0
        # dim["e"][ev.hit_isMasked] = 0
        postfix = f"{i}_unmasked"
    else:
        dim = {k: var[i].to_numpy() for k, var in dict(x=x, y=y, z=z, e=e).items()}
        postfix = f"{i}"

    if remapping:
        dim = channel_remapping(dim)

    fig_2d = plot_slices(pos, dim)
    line = LineFromPlanes(dim)
    line_points = line.get_line_points(pos)
    for ax, (k1, k2) in zip(
        fig_2d.get_axes()[:3], [("x", "y"), ("x", "z"), ("y", "z")]
    ):
        ax.plot(line_points[k1], line_points[k2])
    fig_2d.savefig(img_path / f"2D_slices_{postfix}.png", dpi=300)
    fig_2d.suptitle(i)
    if only_slices:
        return

    fig_3d = plot_3d(dim)
    fig_3d.savefig(img_path / f"3D_{i}.png", dpi=300)

    fig_plotly = plotly_3d(dim)
    # Here we use that the x,y dimensions are similar.
    dxy, dz = [np.linspace(pos[k][0], pos[k][-1], 5) for k in ["x", "z"]]
    mesh_xy, mesh_z = np.meshgrid(dxy, dz)
    dx = line.plane_form(line.res_x.x, mesh_z, mesh_xy)
    dy = line.plane_form(line.res_y.x, mesh_z, mesh_xy)
    fig_plotly.add_surface(x=mesh_xy, y=dy, z=mesh_z, name="plane 1", opacity=0.2)
    fig_plotly.add_surface(x=dx, y=mesh_xy, z=mesh_z, name="plane 2", opacity=0.2)
    pts = line.get_line_points(pos)
    fig_plotly.add_scatter3d(
        x=pts["x"], y=pts["y"], z=pts["z"], name="line fit", mode="lines"
    )
    fig_plotly.update_layout(scene=dict(zaxis=dict(range=[pos["z"][0], pos["z"][-1]])))
    fig_plotly.show()
    fig_plotly.write_html(img_path / f"3D_{postfix}.html")

    return fig_plotly

In [ ]:
do_plotting(
    24,
    # unmasked=True,
    remapping=True,
)

## Mass production of the 2D slice plots

In [ ]:
sys.stdout = open(os.devnull, "w")
for i in tqdm.trange(len(events)):
    do_plotting(
        i,
        # unmasked=True,
        remapping=True,
        only_slices=True,
    )
    plt.close()
sys.stdout = sys.__stdout__